# Exercise 16


In [4]:
import pandas as pd
train_df = pd.read_csv('14_mashable_train_df.csv', index_col=0)
train_df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2013/09/16/sprints-upgrade...,479,11,1045,0.407946,1,0.617021,29,3,1,...,0.033333,1.0,-0.305139,-0.700000,-0.125000,0.0,0.0,0.5,0.0,1
1,http://mashable.com/2014/12/06/eric-garner-pro...,31,14,0,0.000000,0,0.000000,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,-0.5,0.5,0.5,0
2,http://mashable.com/2013/06/06/human-scale-inv...,581,6,604,0.470000,1,0.675516,9,0,1,...,0.100000,1.0,-0.215873,-0.666667,-0.071429,0.1,0.0,0.4,0.0,0
3,http://mashable.com/2014/11/24/beyonce-spotify...,44,16,708,0.474286,1,0.675325,5,2,2,...,0.062500,0.5,-0.376852,-0.875000,-0.100000,0.4,-0.2,0.1,0.2,0
4,http://mashable.com/2013/03/03/superheroic-let...,676,7,30,1.000000,1,1.000000,2,1,0,...,0.250000,0.7,0.000000,0.000000,0.000000,0.0,0.0,0.5,0.0,1


In [5]:
train_df.shape

(23786, 61)

In [6]:
X = train_df.drop(['url', 'Popular'], axis=1)
y = train_df['Popular'].values

# Exercice 16.1 (2 points)

Estimate a Logistic Regression, GaussianNB, K-nearest neighbors and a Decision Tree **Classifiers**

Evaluate using the following metrics:
* Accuracy
* F1-Score
* F_Beta-Score (Beta=2)

Comment about the results

Combine the classifiers and comment

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

models = {'lr': LogisticRegression(),
          'dt': DecisionTreeClassifier(),
          'nb': GaussianNB(),
          'nn': KNeighborsClassifier()}

In [8]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

for model in models.keys():
    models[model].fit(X_train, y_train)
    

In [9]:
# predict test for each model
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [10]:
# Evaluate each model
import numpy as np
from sklearn.metrics import mean_squared_error

for model in models.keys():
    print(model,np.sqrt(mean_squared_error(y_pred[model], y_test)))

nb 0.393959775474
dt 0.456995400081
nn 0.371554571673
lr 0.35772011289


In [11]:
y_pred.head()

,nb,dt,nn,lr
4754,0,0,0,0
10360,0,0,0,0
13048,0,0,0,0
1912,1,0,0,0
3323,0,0,0,0


In [12]:
from sklearn import metrics
for model in models.keys():
    print(model, 'precision',metrics.precision_score(y_pred[model],y_test))
    print(model, 'recall', metrics.recall_score(y_pred[model],y_test))
    print(model, 'f1.score',metrics.f1_score(y_pred[model],y_test))
    print(model, 'fbeta',metrics.fbeta_score(y_pred[model],y_test,beta=4))

nb precision 0.104635761589
nb recall 0.242331288344
nb f1.score 0.146160962072
nb fbeta 0.224920448836
dt precision 0.230463576159
dt recall 0.208383233533
dt f1.score 0.218867924528
dt fbeta 0.209564293305
nn precision 0.0331125827815
nn recall 0.215517241379
nn f1.score 0.0574052812859
nn fbeta 0.162772883953
lr precision 0.00397350993377
lr recall 0.25
lr f1.score 0.00782268578879
lr fbeta 0.0538542766631


# Exercice 16.2

Apply random-undersampling with a target percentage of 0.5

how does the results change

In [14]:
def UnderSampling(X, y, target_percentage=0.5, seed=None):
    # Assuming minority class is the positive
    n_samples = y.shape[0]
    n_samples_0 = (y == 0).sum()
    n_samples_1 = (y == 1).sum()

    n_samples_0_new =  n_samples_1 / target_percentage - n_samples_1
    n_samples_0_new_per = n_samples_0_new / n_samples_0

    filter_ = y == 0

    np.random.seed(seed)
    rand_1 = np.random.binomial(n=1, p=n_samples_0_new_per, size=n_samples)
    
    filter_ = filter_ & rand_1
    filter_ = filter_ | (y == 1)
    filter_ = filter_.astype(bool)
    
    return X[filter_], y[filter_]

In [15]:
X_u, y_u = UnderSampling(X_train, y_train,0.5,666)

for model in models.keys():
    models[model].fit(X_u, y_u)
    
# predict test for each model
y_pred = pd.DataFrame(index=X_test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)
    

from sklearn import metrics
for model in models.keys():
    print(model, 'fbeta',metrics.fbeta_score(y_pred[model],y_test,beta=2))

nb fbeta 0.216116085211
dt fbeta 0.190272067186
nn fbeta 0.193039443155
lr fbeta 0.241096753861


# Exercice 16.3 (3 points)

For each model estimate a BaggingClassifier of 100 models using the under sampled datasets

In [27]:
from sklearn.tree import DecisionTreeRegressor

treereg = DecisionTreeRegressor(max_depth=None, random_state=123)


from sklearn.ensemble import BaggingRegressor
bagreg = BaggingRegressor(DecisionTreeRegressor(), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)

In [28]:
# fit and predict
bagreg.fit(X_train, y_train)
y_pred = bagreg.predict(X_test)
y_pred

array([ 0.25,  0.  ,  0.25, ...,  0.05,  0.09,  0.27])

# Exercice 16.4 (2 points)

Using the under-sampled dataset

Evaluate a RandomForestClassifier and compare the results

change n_estimators=100, what happened